# 빅데이터 EC2코드
### 10분 마다 센서데이터 저장
데이터 형식 : "time : 2021-11-18 13:00:00 / dis : 45 / bed id : 25 / sensoer id : 1"  
1. subscriber로 10분마다 데이터(센서종류, bed_id, 시간, 거리)를 받는다. (10분이라고 하지만 통신 시간까지 고려한다면 1~2초정도 차이가 있는데 이걸 보정)
2. 시간을 10분단위로 정리한다.
3. 2개의 거리를 조합해 앉아(sit, lay, empty)상태를 판별한다.
4. elder_status테이블의 today_status열에 ("sit", "lay", "empty")중 하나를 입력하고 sit, lay, empty열 중 해당 열의 count(int)값을 늘린다.

### 오후 11시 55분 모든 정보 update
1. elder_status테이블의 max_status열에 elder_status테이블의 sit, lay, empty열 중 가장 높은 count를 찾아서 ("sit", "lay", "empty")중 하나를 저장합니다.(만약 동일 count일때 recent_status를 저장합니다.)
2. elder_status테이블의 recent_status열에 today_status값을 저장합니다.

In [2]:
# 라이브러리
import pandas as pd
import mariadb
import time

# DB 관련 전역변수
db_ip = 'multi-big-mariadb.cux0ffgbppdt.ap-northeast-1.rds.amazonaws.com'
db_port = 3306
db_user = 'lab14'
db_passwd = 'multi1234!'
db_name = 'unicodb14'
# mqtt 관련 전역변수
mqtt_ip = "18.221.2.22"
mqtt_port = 22
mqtt_topic = "iot"

# 실험1-1 : 데이터를 받는 즉시 DB저장(센서가 2개일때)

In [18]:
arrData = pd.DataFrame(columns=["time", "dis", "bed_id","sensor_id"])

In [3]:
display(arrData)

,time,dis,bed_id,sensor_id
0,17:10:00,56,1,1


In [16]:
"""
두 센서의 데이터를 받았을때 상태("sit", "lay", "empty")를 구하는 함수
"""
def findStatus(dis1, dis2):
    # 천장에서 침대까지의 거리 + 5
    empty_min1 = 200
    # 천장에서 누웠을때 머리까지의 거리 + 5
    lay_min1 = 180
    # 침대프레임과 벽 사이의 거리
    lay_min1 = 200

    # 판단
    if dis1 > empty_min1:
        if dis2 < lay_min1:
            return "sit"
        return "empty"
    elif dis1 > lay_min1:
        return "lay"
    else:
        return "sit"       

In [ ]:
# rowdata = "dis : 200 / bedID : 1 / sensor_id : 1"
# rowdata = "dis : 200 / bedID : 1 / sensor_id : 2"
# rowdata = "dis : 210 / bedID : 2 / sensor_id : 1"
rowdata = "dis : 49 / bedID : 1 / sensorID : 3"
rowdata = rowdata.split(" / ")
timedata = time.strftime('%H:%M', time.localtime(time.time()))
timedata = timedata[:4] + "0:00"
dis = int(rowdata[0][6:])
bed_id = int(rowdata[1][8:])
sensor_id = int(rowdata[2][11:])

# 같은 침대에서 온 다른 센서 데이터가 있는 지 확인하기
arrived = arrData[arrData["bed_id"] == bed_id]
if len(arrived):
    # 미리온 데이터가 있을때
    if arrived.iloc[0]["sensor_id"] == 1:
        # 상태 구하기
        status = findStatus(arrived.iloc[0]["dis"], dis)
    else:
        status = findStatus(dis, arrived.iloc[0]["dis"])
    print(status, timedata)
    # elder_status 테이블에 저장
    conn = mariadb.connect(host=db_ip, port=db_port, user=db_user, passwd=db_passwd, db=db_name)
    cur = conn.cursor()
    # today_status 변경
    sql = f"UPDATE elder_status SET today_status = '{status}' WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id}) AND time = '{timedata}';"
    cur.execute(sql)
    # count변경
    sql = f"UPDATE elder_status SET {status} = (SELECT {status} FROM elder_status WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id})) + 1 WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id}) AND time = '{timedata}';"
    cur.execute(sql)
    conn.commit()
    conn.close()
    # arrDate에서 데이터 삭제하기
    arrData.drop(arrived.iloc[[0]].index, inplace = True)
else:
    # 미리 온 데이터가 없을때 arrdata에 저장
    arrData = arrData.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)

# 실험1-2 : 데이터를 받는 즉시 DB저장(센서가 3개일때) warning : 세개의 센서를 모두 10분 안에 설치해야합니다.
125 : 누웠을때(110 ~ 140)  
143 : 없을때(140 ~ )  
63~ 64 : 앉을때(50 ~ 110)  
outlier : (~ 49)(800 ~ )

센서1 : 머리쪽 센서  
센서2 : 배쪽 센서  
센서3 : 다리쪽 센서  

앉아있을때 : 센서1,2,3 중 하나라도 앉아있다 이면 앉아 있다.  
누워있을때 : 센서2가 누워있다 일 때 누워있다.   
비여있을대 : 센서1,2,3 모두가 비여있다면 비여 있음  
나머지 예외상황(이불 포함) : 비여있음  

In [21]:
"""
세개 센서의 데이터를 받았을때 상태("sit", "lay", "empty")를 구하는 함수
"""
def findStatus(dis1, dis2, dis3):
    disList = [dis1, dis2, dis3]
    for i in range(3):
        if disList[i] > 800:
            disList[i] = "outlier"
        elif disList[i] >= 140:
            disList[i] = "empty"
        elif disList[i] >= 110:
            disList[i] = "lay"
        elif disList[i] >= 50:
            disList[i] = "sit"
        else :
            disList[i] = "outlier"
    print(disList)
    # 센서2가 "lay"이면 "lay"
    if disList[1] == "lay":
        return "lay"
    elif "sit" in disList:
        return "sit"
    elif len([i for i, value in enumerate(disList) if value == "outlier"]) > 0 and "lay" in disList:
        return "lay"
    else:
        return "empty"

In [ ]:
# rowdata = "dis : 200 / bedID : 1 / sensor_id : 1"
# rowdata = "dis : 200 / bedID : 1 / sensor_id : 2"
# rowdata = "dis : 210 / bedID : 2 / sensor_id : 1"
rowdata = "dis : 49 / bedID : 1 / sensorID : 3"
rowdata = rowdata.split(" / ")
timedata = time.strftime('%H:%M', time.localtime(time.time()))
timedata = timedata[:4] + "0:00"
dis = int(rowdata[0][6:])
bed_id = int(rowdata[1][8:])
sensor_id = int(rowdata[2][11:])

# 같은 침대에서 온 다른 센서 데이터가 있는 지 확인하기
arrived = arrData[arrData["bed_id"] == bed_id]
if len(arrived) == 2:
    # 미리온 센서 데이터가 2개 있을때 상태구해서 DB에 저장하기
    # 상태 판단하기
    arrived = arrived.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)
    arrived.sort_values(by='sensor_id', inplace = True)
    status = findStatus(arrived.iloc[0]["dis"], arrived.iloc[0]["dis"], arrived.iloc[0]["dis"])
    # elder_status 테이블에 저장
    # DB연결
    conn = mariadb.connect(host=db_ip, port=db_port, user=db_user, passwd=db_passwd, db=db_name)
    cur = conn.cursor()
    # today_status 변경
    sql = f"UPDATE elder_status SET today_status = '{status}' WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id}) AND time = '{timedata}';"
    cur.execute(sql)
    # count변경
    sql = f"UPDATE elder_status SET {status} = (SELECT {status} FROM elder_status WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id})) + 1 WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id}) AND time = '{timedata}';"
    cur.execute(sql)
    conn.commit()
    conn.close()
    # arrDate에서 데이터 삭제하기
    arrData.drop(arrived.iloc[[0,1,2]].index, inplace = True)
else:
    # 센서 데이터가 부족할때(도착한것까지 3개가 되지 않았을때) 도착한 데이터 arrived에 넣기
    if len(arrived[arrived["sensor_id"] == sensor_id]) == 0:
        # 동일한 센서의 데이터가 아니면 arrived에 넣기
        arrData = arrData.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)

# 실험2 : mqtt로 정보를 받으면 바로 바로 처리하는 코드 (오류 있는 코드 : 같은 센서값이 한번더 들어와도 DB에 저장됩니다.)

In [4]:
# 라이브러리
import pandas as pd
import mariadb
import paho.mqtt.client as mqtt
import time

# DB 관련 전역변수
db_ip = 'multi-big-mariadb.cux0ffgbppdt.ap-northeast-1.rds.amazonaws.com'
db_port = 3306
db_user = 'lab14'
db_passwd = 'multi1234!'
db_name = 'unicodb14'
# mqtt 관련 전역변수
mqtt_ip = "35.76.212.7"
mqtt_port = 8900
mqtt_topic = "iot"
# 2개센서데이터를 모으기 위한 임시저장소
arrData = pd.DataFrame(columns=["time", "dis", "bed_id","sensor_id"])

# 브로커 접속 시도 결과 처리 콜백 함수
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+ str(rc))
    if rc == 0:
        print("연결 완료")
        client.subscribe(mqtt_topic)
    else:
        print("연결 실패: ", rc)

# 관련 토픽 메시지 수신 콜백 함수
def on_message(client, userdata, msg):
    global arrData
    rowdata = msg.payload.decode()
    print(rowdata)
    # 도착한 데이터를 나누기
    rowdata = rowdata.split(" / ")
    timedata = time.strftime('%H:%M', time.localtime(time.time()))
    timedata = timedata[:4] + "0:00"
    dis = int(rowdata[0][6:])
    bed_id = int(rowdata[1][8:])
    sensor_id = int(rowdata[2][11:])

    # 같은 침대에서 온 다른 센서 데이터가 있는 지 확인하기
    arrived = arrData[arrData["bed_id"] == bed_id]
    if len(arrived):
        # 미리온 데이터가 있을때
        if arrived.iloc[0]["sensor_id"] == 1:
            # 상태 구하기
            status = findStatus(arrived.iloc[0]["dis"], dis)
        else:
            status = findStatus(dis, arrived.iloc[0]["dis"])
        print(status, timedata)
        # elder_status 테이블에 저장
        conn = mariadb.connect(host=db_ip, port=db_port, user=db_user, passwd=db_passwd, db=db_name)
        cur = conn.cursor()
        # today_status 변경
        sql = f"UPDATE elder_status SET today_status = '{status}' WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id}) AND time = '{timedata}';"
        cur.execute(sql)
        # print(sql)
        # count변경
        sql = f"UPDATE elder_status SET {status} = (SELECT {status} FROM elder_status WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id})) + 1 WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id}) AND time = '{timedata}';"
        cur.execute(sql)
        # print(sql)
        conn.commit()
        conn.close()
        # arrDate에서 데이터 삭제하기
        arrData.drop(arrived.iloc[[0]].index, inplace = True)
    else:
        # 미리 온 데이터가 없을때 arrdata에 저장
        arrData = arrData.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)

"""
두 센서의 데이터를 받았을때 상태("sit", "lay", "empty")를 구하는 함수
"""
def findStatus(dis1, dis2):
    # 천장에서 침대까지의 거리 + 5
    empty_min1 = 200
    # 천장에서 누웠을때 머리까지의 거리 + 5
    lay_min1 = 180
    # 침대프레임과 벽 사이의 거리
    lay_min1 = 200

    # 판단
    if dis1 > empty_min1:
        if dis2 < lay_min1:
            return "sit"
        return "empty"
    elif dis1 > lay_min1:
        return "lay"
    else:
        return "sit"       


# 1. MQTT 클라이언트 객체 인스턴스화
client = mqtt.Client()
# 2. 관련 이벤트에 대한 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message

try :
    # 3. 브로커 연결
    client.connect(mqtt_ip, port=mqtt_port, keepalive=65)
    # 4. 메시지 루브 - 이벤트 발생시 해당 콜백 함수 호출됨
    client.loop_forever()
except Exception as err:
    print('에러 : %s'%err)

Connected with result code 0
연결 완료
dis : 52 / bedID : 1 / sensorID : 1
dis : 53 / bedID : 1 / sensorID : 1
sit 17:20:00
에러 : Lock wait timeout exceeded; try restarting transaction


# 실험2-2 : mqtt로 데이터를 받는 즉시 DB저장(센서가 3개일때) warning : 세개의 센서를 모두 10분 안에 설치해야합니다.

In [4]:
# 라이브러리
import pandas as pd
import mariadb
import paho.mqtt.client as mqtt
import time

# DB 관련 전역변수
db_ip = 'multi-big-mariadb.cux0ffgbppdt.ap-northeast-1.rds.amazonaws.com'
db_port = 3306
db_user = 'lab14'
db_passwd = 'multi1234!'
db_name = 'unicodb14'
# mqtt 관련 전역변수
mqtt_ip = "35.76.212.7"
mqtt_port = 8900
mqtt_topic = "iot"
# 2개센서데이터를 모으기 위한 임시저장소
arrData = pd.DataFrame(columns=["time", "dis", "bed_id","sensor_id"])

# 브로커 접속 시도 결과 처리 콜백 함수
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+ str(rc))
    if rc == 0:
        print("연결 완료")
        client.subscribe(mqtt_topic)
    else:
        print("연결 실패: ", rc)

# 관련 토픽 메시지 수신 콜백 함수
def on_message(client, userdata, msg):
    global arrData
    rowdata = msg.payload.decode()
    print(rowdata)
    # 도착한 데이터를 나누기
    rowdata = rowdata.split(" / ")
    timedata = time.strftime('%H:%M', time.localtime(time.time()))
    timedata = timedata[:4] + "0"
    dis = int(rowdata[0][6:])
    bed_id = int(rowdata[1][8:])
    sensor_id = int(rowdata[2][11:])

    # 같은 침대에서 온 다른 센서 데이터가 있는 지 확인하기
    arrived = arrData[arrData["bed_id"] == bed_id]
    if len(arrived) == 2:
        # 미리온 센서 데이터가 2개 있을때 상태구해서 DB에 저장하기
        # 상태 판단하기
        arrived = arrived.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)
        arrived.sort_values(by='sensor_id', inplace = True)
        status = findStatus(arrived.iloc[0]["dis"], arrived.iloc[0]["dis"], arrived.iloc[0]["dis"])
        # elder_status 테이블에 저장
        # DB연결
        conn = mariadb.connect(host=db_ip, port=db_port, user=db_user, passwd=db_passwd, db=db_name)
        cur = conn.cursor()
        # today_status 변경
        sql = f"UPDATE elder_status SET today_status = '{status}' WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id}) AND time = '{timedata}';"
        cur.execute(sql)
        print(sql)
        # count변경
        sql = f"UPDATE elder_status SET {status} = (SELECT {status} FROM elder_status WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id})) + 1 WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = {bed_id}) AND time = '{timedata}';"
        cur.execute(sql)
        print(sql)
        conn.commit()
        conn.close()
        print("DB updata 완료")
        # arrDate에서 데이터 삭제하기
        arrData.drop(arrived.iloc[[0,1]].index, inplace = True)
    else:
        # 센서 데이터가 부족할때(도착한것까지 3개가 되지 않았을때) 도착한 데이터 arrived에 넣기
        if len(arrived[arrived["sensor_id"] == sensor_id]) == 0:
            # 동일한 센서의 데이터가 아니면 arrived에 넣기
            arrData = arrData.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)

"""
세개 센서의 데이터를 받았을때 상태("sit", "lay", "empty")를 구하는 함수
"""
def findStatus(dis1, dis2, dis3):
    disList = [dis1, dis2, dis3]
    for i in range(3):
        if disList[i] > 800:
            disList[i] = "outlier"
        elif disList[i] >= 140:
            disList[i] = "empty"
        elif disList[i] >= 110:
            disList[i] = "lay"
        elif disList[i] >= 50:
            disList[i] = "sit"
        else :
            disList[i] = "outlier"
    print(disList)
    # 센서2가 "lay"이면 "lay"
    if disList[1] == "lay":
        return "lay"
    elif "sit" in disList:
        return "sit"
    elif len([i for i, value in enumerate(disList) if value == "outlier"]) > 0 and "lay" in disList:
        return "lay"
    else:
        return "empty" 


# 1. MQTT 클라이언트 객체 인스턴스화
client = mqtt.Client()
# 2. 관련 이벤트에 대한 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message

try :
    # 3. 브로커 연결
    client.connect(mqtt_ip, port=mqtt_port, keepalive=65)
    # 4. 메시지 루브 - 이벤트 발생시 해당 콜백 함수 호출됨
    client.loop_forever()
except Exception as err:
    print('에러 : %s'%err)

Connected with result code 0
연결 완료
dis : 21 / bedID : 1 / sensorID : 3
dis : 8 / bedID : 1 / sensorID : 1
dis : 21 / bedID : 1 / sensorID : 3
['outlier', 'outlier', 'outlier']
UPDATE elder_status SET today_status = 'empty' WHERE elder_id = (SELECT elder_id FROM elder WHERE bed_id = 1) AND time = '11:10';
에러 : Subquery returns more than 1 row


# 실험3 : 11시55분에 elder_status테이블을 갱신하는 코드
elder_status테이블의 max_status열에 elder_status테이블의 sit, lay, empty열 중 가장 높은 count를 찾아서 ("sit", "lay", "empty")중 하나를 저장합니다.(만약 동일 count일때 recent_status를 저장합니다.)  
elder_status테이블의 recent_status열에 today_status값을 저장합니다.  
https://blog.naver.com/PostView.nhn?blogId=shino1025&logNo=221432633410&redirect=Dlog&widgetTypeCall=true&directAccess=false

In [25]:
# 라이브러리
import pandas as pd
import mariadb

# DB 관련 전역변수
db_ip = 'multi-big-mariadb.cux0ffgbppdt.ap-northeast-1.rds.amazonaws.com'
db_port = 3306
db_user = 'lab14'
db_passwd = 'multi1234!'
db_name = 'unicodb14'

# elder_status 테이블에 저장
conn = mariadb.connect(host=db_ip, port=db_port, user=db_user, passwd=db_passwd, db=db_name)
cur = conn.cursor()
# elder_status 테이블 가져오기 
sql = f"SELECT * from elder_status"
cur.execute(sql)
result = cur.fetchall()
df = pd.DataFrame(result)
df.columns = ["status_id", "elder_id", "time", "lay", "sit", "empty", "recent_status", "today_status", "max_status"]
conn.close()

# elder_status 테이블변경
conn = mariadb.connect(host=db_ip, port=db_port, user=db_user, passwd=db_passwd, db=db_name)
cur = conn.cursor()
for row in range(len(df)):
    # max_status열 변경
    dic = {'lay' : df.loc[row, "lay"], "sit" : df.loc[row, "sit"], "empty" : df.loc[row, "empty"]}
    max_status = sorted(dic.items(), key=(lambda x:x[1]), reverse = True)[0][0]
    sql = f"UPDATE elder_status SET max_status = '{max_status}' WHERE status_id = {df.loc[row, 'status_id']};"
    cur.execute(sql)
    # recent_status열 변경
    sql = f"UPDATE elder_status SET recent_status = '{df.loc[row, 'today_status']}' WHERE status_id = {df.loc[row, 'status_id']};"
    cur.execute(sql)
    # today_status열 변경
    sql = f"UPDATE elder_status SET today_status = NULL WHERE status_id = {df.loc[row, 'status_id']};"
    cur.execute(sql)
conn.commit()
conn.close()

# 실험5 : DB insert

In [13]:
import mariadb
import random

# DB 관련 전역변수
db_ip = 'multi-big-mariadb.cux0ffgbppdt.ap-northeast-1.rds.amazonaws.com'
db_port = 3306
db_user = 'lab14'
db_passwd = 'multi1234!'
db_name = 'unicodb14'
elder_id = 2

conn = mariadb.connect(host=db_ip, port=db_port, user=db_user, passwd=db_passwd, db=db_name)
cur = conn.cursor()

for h in range(24):
    for m in range(0, 60, 10):
        sql = f"INSERT INTO elder_status (elder_id, time, lay, sit, empty) VALUES ({elder_id}, '{h:02d}:{m:02d}', {random.randint(0, 100)}, {random.randint(0, 100)}, {random.randint(0, 100)});"
        cur.execute(sql)

conn.commit()
conn.close()

# 실험 6 : DB updata REST API로 변경

In [ ]:
# 라이브러리
import pandas as pd
import requests
import json
import paho.mqtt.client as mqtt
import time

# mqtt 관련 전역변수
mqtt_ip = "35.76.212.7"
mqtt_port = 8900
mqtt_topic = "iot"
# 2개센서데이터를 모으기 위한 임시저장소
arrData = pd.DataFrame(columns=["time", "dis", "bed_id","sensor_id"])

# 브로커 접속 시도 결과 처리 콜백 함수
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+ str(rc))
    if rc == 0:
        print("연결 완료")
        client.subscribe(mqtt_topic)
    else:
        print("연결 실패: ", rc)

# 관련 토픽 메시지 수신 콜백 함수
def on_message(client, userdata, msg):
    global arrData
    rowdata = msg.payload.decode()
    print(rowdata)
    # 도착한 데이터를 나누기
    rowdata = rowdata.split(" / ")
    timedata = time.strftime('%H:%M', time.localtime(time.time()))
    timedata = timedata[:4] + "0"
    dis = int(rowdata[0][6:])
    bed_id = int(rowdata[1][8:])
    sensor_id = int(rowdata[2][11:])

    # 같은 침대에서 온 다른 센서 데이터가 있는 지 확인하기
    arrived = arrData[arrData["bed_id"] == bed_id]
    if len(arrived) == 2:
        # 미리온 센서 데이터가 2개 있을때 상태구해서 DB에 저장하기
        # 상태 판단하기
        arrived = arrived.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)
        arrived.sort_values(by='sensor_id', inplace = True)
        status = findStatus(arrived.iloc[0]["dis"], arrived.iloc[0]["dis"], arrived.iloc[0]["dis"])
        # elder_status 테이블에 저장
        headers = {'Content-Type': 'application/json; chearset=utf-8'}
        data_put ={"elder_id":bed_id,"time":timedata,"today_status":status}
        res = requests.put('http://3.134.120.190:8000/api/facility/occupancy/',  data=json.dumps(data_put), headers=headers)
        print("DB updata 완료")
        # arrDate에서 데이터 삭제하기
        arrData.drop(arrived.iloc[[0,1]].index, inplace = True)
    else:
        # 센서 데이터가 부족할때(도착한것까지 3개가 되지 않았을때) 도착한 데이터 arrived에 넣기
        if len(arrived[arrived["sensor_id"] == sensor_id]) == 0:
            # 동일한 센서의 데이터가 아니면 arrived에 넣기
            arrData = arrData.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)

"""
세개 센서의 데이터를 받았을때 상태("sit", "lay", "empty")를 구하는 함수
"""
def findStatus(dis1, dis2, dis3):
    disList = [dis1, dis2, dis3]
    for i in range(3):
        if disList[i] > 800:
            disList[i] = "outlier"
        elif disList[i] >= 140:
            disList[i] = "empty"
        elif disList[i] >= 110:
            disList[i] = "lay"
        elif disList[i] >= 50:
            disList[i] = "sit"
        else :
            disList[i] = "outlier"
    print(disList)
    # 센서2가 "lay"이면 "lay"
    if disList[1] == "lay":
        return "lay"
    elif "sit" in disList:
        return "sit"
    elif len([i for i, value in enumerate(disList) if value == "outlier"]) > 0 and "lay" in disList:
        return "lay"
    else:
        return "empty" 


# 1. MQTT 클라이언트 객체 인스턴스화
client = mqtt.Client()
# 2. 관련 이벤트에 대한 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message

try :
    # 3. 브로커 연결
    client.connect(mqtt_ip, port=mqtt_port, keepalive=65)
    # 4. 메시지 루브 - 이벤트 발생시 해당 콜백 함수 호출됨
    client.loop_forever()
except Exception as err:
    print('에러 : %s'%err)

In [1]:
import requests
import json
bed_id =1
time = "14:00"
status ="lay"
headers = {'Content-Type': 'application/json; chearset=utf-8'}
data_put ={"elder_id":bed_id,"time":time,"today_status":status}
res = requests.put('http://3.134.120.190:8000/api/facility/occupancy/',  data=json.dumps(data_put), headers=headers)

# 실험7 : 테스트

In [ ]:
# 라이브러리
import pandas as pd
import requests
import json
import paho.mqtt.client as mqtt
import time

# mqtt 관련 전역변수
mqtt_ip = "35.76.212.7"
mqtt_port = 8900
mqtt_topic = "iot"
# 2개센서데이터를 모으기 위한 임시저장소
arrData = pd.DataFrame(columns=["time", "dis", "bed_id","sensor_id"])

# 브로커 접속 시도 결과 처리 콜백 함수
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+ str(rc))
    if rc == 0:
        print("연결 완료")
        client.subscribe(mqtt_topic)
    else:
        print("연결 실패: ", rc)

# 관련 토픽 메시지 수신 콜백 함수
def on_message(client, userdata, msg):
    global arrData
    rowdata = msg.payload.decode()
    # print(rowdata)
    # 도착한 데이터를 나누기
    rowdata = rowdata.split(" / ")
    timedata = time.strftime('%H:%M:%S', time.localtime(time.time()))
    timedata = timedata[:7] + "0"
    dis = int(rowdata[0][6:])
    bed_id = int(rowdata[1][8:])
    sensor_id = int(rowdata[2][11:])
    # print(timedata)

    # 같은 침대에서 온 다른 센서 데이터가 있는 지 확인하기
    arrived = arrData[arrData["bed_id"] == bed_id]
    if len(arrived) == 2:
        # 미리온 센서 데이터가 2개 있을때 상태구해서 DB에 저장하기
        # 상태 판단하기
        arrived = arrived.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)
        arrived.sort_values(by='sensor_id', inplace = True)
        status = findStatus(arrived.iloc[0]["dis"], arrived.iloc[1]["dis"], arrived.iloc[2]["dis"])
        # 테스트
        print(status)
        # arrDate에서 데이터 삭제하기
        arrData.drop(arrived.iloc[[0,1]].index, inplace = True)
    else:
        # 센서 데이터가 부족할때(도착한것까지 3개가 되지 않았을때) 도착한 데이터 arrived에 넣기
        if len(arrived[arrived["sensor_id"] == sensor_id]) == 0:
            # 동일한 센서의 데이터가 아니면 arrived에 넣기
            arrData = arrData.append({'time' : timedata , 'dis' : dis, 'bed_id' : bed_id, 'sensor_id' : sensor_id} , ignore_index=True)

"""
세개 센서의 데이터를 받았을때 상태("sit", "lay", "empty")를 구하는 함수
"""
def findStatus(dis1, dis2, dis3):
    disList = [dis1, dis2, dis3]
    # print(disList)
    for i in range(3):
        if disList[i] > 800:
            disList[i] = "outlier"
        elif disList[i] >= 140:
            disList[i] = "empty"
        elif disList[i] >= 110:
            disList[i] = "lay"
        elif disList[i] >= 60:
            disList[i] = "sit"
        else :
            disList[i] = "outlier"
    print(disList)
    # 센서2가 "lay"이면 "lay"
    if disList[1] == "lay":
        return "lay"
    elif "sit" in disList:
        return "sit"
    elif len([i for i, value in enumerate(disList) if value == "outlier"]) > 0 and "lay" in disList:
        return "lay"
    else:
        return "empty" 


# 1. MQTT 클라이언트 객체 인스턴스화
client = mqtt.Client()
# 2. 관련 이벤트에 대한 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message

try :
    # 3. 브로커 연결
    client.connect(mqtt_ip, port=mqtt_port, keepalive=65)
    # 4. 메시지 루브 - 이벤트 발생시 해당 콜백 함수 호출됨
    client.loop_forever()
except Exception as err:
    print('에러 : %s'%err)

Connected with result code 0
연결 완료
['empty', 'empty', 'empty']
empty
['empty', 'sit', 'sit']
sit
['empty', 'sit', 'lay']
sit
['empty', 'empty', 'lay']
empty
['empty', 'lay', 'sit']
lay
['empty', 'sit', 'empty']
sit
['empty', 'sit', 'empty']
sit
['empty', 'sit', 'lay']
sit
['empty', 'sit', 'empty']
sit
['empty', 'lay', 'sit']
lay
['empty', 'sit', 'empty']
sit
['empty', 'lay', 'sit']
lay
['empty', 'empty', 'lay']
empty
['empty', 'sit', 'sit']
sit
['empty', 'sit', 'lay']
sit
['empty', 'sit', 'empty']
sit
['empty', 'empty', 'outlier']
empty
['empty', 'empty', 'empty']
empty
['empty', 'empty', 'empty']
empty
['empty', 'empty', 'empty']
empty
['empty', 'sit', 'empty']
sit
['empty', 'outlier', 'empty']
empty
['empty', 'empty', 'empty']
empty
['empty', 'lay', 'empty']
lay
['empty', 'empty', 'empty']
empty
['empty', 'empty', 'sit']
sit
['empty', 'sit', 'empty']
sit
['empty', 'empty', 'empty']
empty
['empty', 'empty', 'empty']
empty
['empty', 'sit', 'empty']
sit
['empty', 'empty', 'empty']
empty